This Notebook was written by [Ezer Karash](https://www.linkedin.com/in/ezer-karash-205a5746). It's totaly open, just do me a favor and mention my name.

Pandas is a set of Data tools for Python, originaly wriitten by [Wes McKinney](http://wesmckinney.com/).

## Table of contents

* [Importing](#importing)
* [Creating a DataFrame](#creating_a_dataframe)
    * [Manually create a DF](#manually)
    * [Create a DF from csv](#from_csv)
    * [Create a DF from Json](#from_json)
* [DF building blocks (Series & Arrays)](#internals)
* [Accessing Data](#access)
* [Data Manipulation](#manipulation)
    * [Add Column](#add)
    * [Delete column](#del)
    * [Perfom calculation on columns](#cal)
* [Groupby](#group)
    * [By one column](#group1)
    * [By two columns](#group2)
    * [Apply custom function to groups (Split, Apply,Combine)](#apply)
    * [Filter groups](#filter) 
* [Descriptives](#desc)
* [Filter DF's](#filter_df)
* [Reset Index](#reset)
* [Merge DF's](#merge)
    * [Inner Join](#inner)
    * [Outer Join](#outer) 
    * [Left Join](#left)
* [Crosstabs](#cross)
* [Output](#out)
    * [Output DF to CSV](#to_csv)
    * [Output DF to Excel](#to_excel) 
* [Exercise](#exer)

## Importing

In [5]:
import numpy as np
import pandas as pd

import os
#os.chdir('data_files') #change working directory 

import matplotlib.pyplot as plt
#make graphs appear in the notebook:
%matplotlib inline 

<a id='creating_a_dataframe'></a>

## Creating a DataFrame

The basic object in PANDAS is the DataFrame which is a table with column headers and an index.

There are a couple ways to create a DF:

<a id='manually'></a>

### Manually create a DF

In [6]:
M = pd.DataFrame ([[1,2,3],[4,5,6],[7,8,9]])
M

,0,1,2
0,1,2,3
1,4,5,6
2,7,8,9


In [1]:
M.plot()

NameError: name 'M' is not defined

<a id='from_csv'></a>

### Create a DF from csv

In [ ]:
apt_prices = pd.read_csv('apt_prices.csv')
apt_prices

<a id='from_json'></a>

### Create a DF from Json

Create a Json file from the DF previously created: 

In [ ]:
M.to_json('m.json')

view the Json (this is plain python):

In [ ]:
with open('m.json') as f:
    print f.readlines()

create a DF from the Json file:

In [ ]:
mm = pd.read_json('m.json')

In [ ]:
mm

<a id='internals'></a>

## DF building blocks (Series & Arrays)

Each Column in the DataFrame is a Pandas Series Object, which could be created on its own:

In [9]:
S = pd.Series([1,2,3,'4','5','A','b','cat','dog',7.0,8.1])

In [10]:
S

0       1
1       2
2       3
3       4
4       5
5       A
6       b
7     cat
8     dog
9       7
10    8.1
dtype: object

And then a couple of Series could be binded into a DF:

In [11]:
S_doubled = pd.Series([x*2 for x in S])
S_df = pd.DataFrame([S,S_doubled],index=['Single','Double'])
S_df

,0,1,2,3,4,5,6,7,8,9,10
Single,1,2,3,4,5,A,b,cat,dog,7.0,8.1
Double,2,4,6,44,55,AA,bb,catcat,dogdog,14.0,16.2


Than the DF could be transposed, if needed:

In [12]:
S_df.T

,Single,Double
0,1,2
1,2,4
2,3,6
3,4,44
4,5,55
5,A,AA
6,b,bb
7,cat,catcat
8,dog,dogdog
9,7,14


Notice, that in the DF print out does not have a string/numeral distinction, BE CAREFULL!!!

A Series is an Indexed (labeled) Numpy one dimensional array:

In [13]:
S.index

RangeIndex(start=0, stop=11, step=1)

In [14]:
S.values

array([1, 2, 3, '4', '5', 'A', 'b', 'cat', 'dog', 7.0, 8.1], dtype=object)

In [15]:
type(S.index) , type(S.values)

(pandas.indexes.range.RangeIndex, numpy.ndarray)

hence a DF is an array of arrays with index:

In [ ]:
S_df.index

In [ ]:
S_df.values

In [ ]:
type(S_df.index),type(S_df.values[0]),type(S_df.values[0][0])

Guess what happends if you Transpose:

In [ ]:
S_df.T.index

In [ ]:
S_df.T.values

<a id='access'></a>

## Accessing Data

You can access columns by name (remember a single column is a series):

In [16]:
apt_prices ['Jerusalem']

NameError: name 'apt_prices' is not defined

or by access location:

In [ ]:
apt_prices.ix[:,2] #all rows, second column

you can access a single cell too:

In [ ]:
apt_prices ['Jerusalem'][0]

and ranges as well:

In [ ]:
apt_prices.ix[0:3,2]

<a id='manipulation'></a>

## Data Manipulation

### Change columns name:

In [ ]:
apt_prices.rename(columns = {'Unnamed: 0':'Year'},inplace=True)
apt_prices

<a id='add'></a>

#### Add Column

In [ ]:
apt_prices['Center-North'] = apt_prices.Center-apt_prices.North
print apt_prices.shape
apt_prices

<a id='del'></a>

#### Delete column

In [ ]:
apt_prices.drop('Center-North',axis=1,inplace=True)
apt_prices.shape

<a id='cal'></a>

#### Perfom calculation on columns

Multiply by 1000 and round:

In [ ]:
apt_prices.ix[:,1:] = np.round(apt_prices.ix[:,1:]*1000,0)
apt_prices

<a id='group'></a>

## Groupby

<a id='group1'></a>

### By One Column

Add a decade column:


In [ ]:
decade = []
for year in apt_prices['Year']:
    if int(year[:4])<2011:
        decade.append(2000)
    else:
        decade.append(2010)
apt_prices['Decade'] = decade
apt_prices.ix[[0,3],:]

For performance reasons, it is good to store the grouping object as variable and not regroup the DF every time you need an aggragation calculation:

In [ ]:
grouped_by_decades = apt_prices.groupby('Decade')

What was the highest price every decade?

In [ ]:
apt_prices.groupby('Decade').max()

What was the lowest price every decade?

In [ ]:
apt_prices.groupby('Decade').min()

<a id='group2'></a>

### By 2 columns

Load cities geo spatial Data

In [ ]:
cities = pd.read_csv('cities_long.csv')
cities.head()

Add location Categories

In [ ]:
cities['West/East'] = ['West' if line<0 else 'East' for line in cities['Longitude']]
cities['South/North'] = ['South' if line<0 else 'North' for line in cities['Latitude']]
cities.head()

group:

In [ ]:
quarters = cities.groupby(['West/East','South/North'])

number of elements in each quarter:

In [ ]:
quarters.count()

Notice the Multindex, used as follows:

In [ ]:
quarters.count().ix['East','North']

<a id='apply'></a>

#### Apply custom function to groups (Split, Apply,Combine)

So far we've used ready made functions on the groups (max, min, mean, count etc.), what if we want a made up oporation ran over each group, for instance what is the gap between the max price and min price in evey region at every decade, we can define a function and apply it the grouped object:

In [ ]:
def MinMax(df):
    df = df.drop(['Year','Decade'],axis=1)
    return df.max()-df.min()

In [ ]:
grouped_by_decades.apply(MinMax)

Or in the nameless lambda version:

In [ ]:
grouped_by_decades.apply(lambda x: x.drop(['Year','Decade'],axis=1).max()-x.drop(['Year','Decade'],axis=1).min())

<a id='filter'></a>

### Filter Groups

Keep only cities from world quarters with more than 1M cities:

In [ ]:
df = quarters.filter(lambda x: len(x)>1000000)
print df.shape
df[:7]

<a id='desc'></a>

### Descriptives

In [ ]:
apt_prices.shape

In [ ]:
apt_prices.plot()
plt.show()

In [ ]:
apt_prices.describe()

Which area changed the most?

In [ ]:
change=[]
for i in apt_prices.columns[2:]:
    change.append([round(apt_prices[i][(len(apt_prices)-1)]/apt_prices[i][0],2),i])
sorted(change,reverse=True)

In [ ]:
cities.shape

Five most popular city names:

In [ ]:
cities.City.value_counts() [:5]

How many unique city names?

In [ ]:
cities.City.nunique()

<a id='filter_df'></a>

## Filter DF's

Get only the cities in Japan:

In [ ]:
cities[cities.Country=='jp'][:5]

Get only the cities in Japan north to Longt 140:

In [ ]:
cities[(cities.Country=='jp') & (cities.Longitude>140)][:5]

Get only japanese Citys starting with 'b':

In [ ]:
#create list of Japnese cities starting with 'b':
b_jp_cities = [x for x in cities[cities.Country=='jp'].City if x[0]=='b']
#filter the data according to list:
cities[cities.City.isin(b_jp_cities)][:7]

<a id='reset'></a>

## Reset Index

Pandas always keeps the original indexes (in cases of filteartion for instance), if you want a new index, you have to reset it:

In [ ]:
cities[cities.Region=='CA'][:5]

In [ ]:
cities[cities.Region=='CA'].reset_index()[:5]

In [ ]:
cities[cities.Region=='CA'].reset_index(drop=True)[:5]

<a id='merge'></a>

## Merge DF's

Merge is Pandas eqivalent of SQL's Join and hence could be done in inner, outer or partial mode.

<a id='inner'></a>

### Inner Join

Get apt_prices and city's geo location in case both data exists:

In [ ]:
#transpose 
apt_T = apt_prices.T
#set years as headers
apt_T.columns = apt_prices.Year.values
#put cities in column and set a numerical index:
apt_T_indexed = apt_T.reset_index()
#name joining column (foereign key) with identical names
apt_T_indexed.rename(columns = {'index':'City'},inplace=True)
#lower cases
apt_T_indexed.City = [x.lower() for x in apt_T_indexed.City]

In [ ]:
apt_T_indexed.merge(cities)[:10]

Oops, there are Jerusalems all over the place, and there is a city named Total, try again with Israel only:

In [ ]:
apt_T_indexed.merge(cities[cities.Country=='il'])[:10]

<a id='outer'></a>

### Outer join

Get all cities from tables, give full data where possible:

In [ ]:
apt_T_indexed.merge(cities[cities.Country=='il'],how='outer')[:15]

<a id='left'></a>

### Left Join

Get all cities from the prices table, add geo table data whwere possible:

In [ ]:
apt_T_indexed.merge(cities[cities.Country=='il'],how='left')[:15]

<a id='cross'></a>

## CrossTabs

CrossTabulate 2 series:

In [ ]:
us = cities[(cities.Country=='us')]
#find US 10 most popular city names
pop = us.City.value_counts()[:10].index

In [ ]:
pop

In [ ]:
us =  us[us.City.isin(pop)]

In [ ]:
pd.crosstab(us.Region,us.City)

<a id='out'></a>

## Output

<a id='to_csv'></a>

### Output To CSV

In [ ]:
apt_prices.to_csv('apt_prices_with decades.csv',index=False) #if the index is just runing numbers it is useless

<a id='to_excel'></a>

### Output to excel

This provides format features, and most useful the ability to output a multi sheet excel

In [ ]:
xl_writer = pd.ExcelWriter('cities_by_country.xlsx') #create a writer yo can add to

In [ ]:
cities[cities.Country=='il'].to_excel(xl_writer, sheet_name='Israel')
cities[cities.Country=='it'].to_excel(xl_writer, sheet_name='Italy')

In [ ]:
xl_writer.save()

<a id='exer'></a>

## Exercise

Find the closest cities per region in Israel for regions with less than 100 cities (Hint - look up itertools.combinations).

### Solution

In [ ]:
def Distance(city1,city2,df):
    lat1 = df[df.City==city1].reset_index().Latitude[0]
    lat2 = df[df.City==city2].reset_index().Latitude[0]
    long1 = df[df.City==city1].reset_index().Longitude[0]
    long2 = df[df.City==city2].reset_index().Longitude[0]
    return (lat2-lat1)**2+(long2-long1)**2

In [ ]:
from itertools import combinations as comb

In [ ]:
def Closest(df):
    a_smallest = 'na'
    b_smallest = 'na'
    smallest = []
    for a,b in comb(df.City,2):
        distance = Distance(a,b,df)
        if a_smallest=='na' and b_smallest=='na':
            a_smallest = a
            b_smallest = b
            distance_smallest = distance
        elif distance < distance_smallest:
            a_smallest = a
            b_smallest = b
            distance_smallest = distance
            smallest.append([a_smallest,b_smallest])
        elif distance==distance_smallest:
            smallest.append([a_smallest,b_smallest])
    df_new = pd.DataFrame(smallest)
    if df_new.shape[0]==0:
        df_new = pd.DataFrame([a_smallest,b_smallest])
    df_new.columns = ['City_a','City_b']
    dfr = df.reset_index() #needed for getting first element
    df_new['Country'] = dfr.Country[0]
    df_new['Region'] = dfr.Region[0]
    df_new['Distance'] = distance_smallest
    return df_new

In [ ]:
#leave only Israel
data = cities[cities.Country=='il']
data.shape

In [ ]:
#leave only regions with at least 200 cities:
data = data.groupby('Region').filter(lambda x: x.City.nunique()<=100)
data.shape

In [ ]:
#group:
grouped = data.groupby(['Country','Region'])

In [ ]:
%%time
results = grouped.apply(Closest)

In [ ]:
results